# Next overpass predictor

## Background
* Knowing the time of a satellite overpass (OP) at a precise location is crucial to plan and prepare disaster impact studies. 
The script below can be used to predict the overpasses of the Landsat 8 & 9 and Sentinel 1 & 2 satellites over a  selected location. For Landsat 8 this occurs every 16 days and for Sentinel 2A / 2B this occurs every 10 days.

* The code calls the Python package 'next_pass' located at https://github.com/ehavazli/next_pass. The latter predicts the next overpass of the satellite of interest by scanning the relevant acquisition plans:
	- Landsat acquisition plans (json files) : https://landsat.usgs.gov/sites/default/files/landsat_acq/assets/json/cycles_full.json
	- Sentinel acquisition plans (KML files to import to Google Earth Pro) : https://sentinel.esa.int/web/sentinel/copernicus/sentinel-1/acquisition-plans  

## Description

All what a user needs to provide is the precise location for which he desires to identify the next overpasses. The location can be inputted as (latitude, longitude) or as the name of the city of interest. The script returns the next collect for Sentinel-1 and Sentinel-2 and the next passes, in ascending and descending directions separately, for Landsat-8 and Landsat-9:

- Specify a location 
- Run find_next_overpass for Sentinel-1, Sentinel-2 and the Landsats (8&9) 
- Visualize each of the above predicted overpass 

The outputs of next_pass can be compared against overpasses of the site you are interested in using the ESA Orbital Prediction and Overpass Tool (OPOT) at https://evdc.esa.int/orbit/ 


## Getting started
To run the overpass predictor with the given location, run all cells in the notebook starting with the "Load packages" cell.

### Load packages and functions

In [1]:
import next_pass
import folium
import re  # Import regular expressions
import random  # To generate random colors
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape, box
from geopy.geocoders import Nominatim

In [2]:
import colorsys

# Style function for the bounding box GeoJSON layer
def style_function(feature):
    return {
        'fillColor': '#808080',  # Gray fill color
        'color': '#000000',       # Black border color
        'weight': 4,              # Thicker border (increased thickness)
        'fillOpacity': 0.3        # Fill opacity (adjust if needed)
    }
# Function to generate random hex color
def random_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Function to print text with color in console (ANSI escape code)
def print_colored_text(text, color):
    # Escape sequence for colored text
    print(f"\033[38;2;{color[0]};{color[1]};{color[2]}m{text}\033[39m")

# 
def hsl_distinct_colors(n):
    colors = []
    for i in range(n):
        # Generate colors with different hues
        hue = i / float(n)  # Hue ranges from 0 to 1
        color = colorsys.hsv_to_rgb(hue, 1.0, 1.0)  # Convert HSL to RGB
        # Convert from RGB (0-1) to hex (#RRGGBB)
        rgb = [int(c * 255) for c in color]
        hex_color = "#{:02x}{:02x}{:02x}".format(*rgb)
        colors.append(hex_color)
    return colors

def spread_rgb_colors(n):
    colors = []
    step = 255 // n  # Divide the color space into n parts
    for i in range(n):
        # Spread out the color values across the RGB spectrum
        r = (i * step) % 256
        g = ((i + 1) * step) % 256
        b = ((i + 2) * step) % 256
        hex_color = "#{:02x}{:02x}{:02x}".format(r, g, b)
        colors.append(hex_color)
    return colors

def hsl_distinct_colors_improved(num_colors):
    colors = []
    
    for i in range(num_colors):
        # Set Hue (H) to a random value, excluding extremes like 0° (red) and 60° (yellow)
        hue = (i * 360 / num_colors) % 360
        
        # Set Saturation (S) to a high value (e.g., 70%) for vivid colors
        saturation = random.randint(60, 80)  # Avoid dull colors
        
        # Set Lightness (L) to a lower value to avoid bright, light colors like yellow (range 30-50%)
        lightness = random.randint(30, 50)  # Darker or neutral colors

        # Convert HSL to RGB using the colorsys library
        r, g, b = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)

        # Convert RGB to hex format (RGB values are in [0, 1], so multiply by 255)
        hex_color = "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))
        colors.append(hex_color)
    
    return colors

### Specify location
Start with selecting the location by  precising the latitude/longitude. 

In [3]:
# Locations (here La Crescenta). If overpasses are sought for 1 location, just set min and max params to the same value
lat_S, lat_N, lon_W, lon_E = 34.230429, 34.230429, -118.2350733, -118.2350733

In [4]:
# Users can provide SNEW coordinates to define a bounding box
lat_S, lat_N, lon_W, lon_E = 32, 35, -120, -116

In [5]:
# Users can also use a predefined polygon (kml file) exported from Google Earth, for example
location_file_path = '/Users/ifenni/Desktop/JPL/ARIA_OPERA/Codes/next_pass_dev/LA_National_Forest.kml'

### Specify satellites of interest 
For now, the tool operates for Sentinel 1A and 2A and Landsat 8 and 9.

In [6]:
# Satellites
sat1 = "sentinel-1"
sat2 = "sentinel-2"
sat3 = "landsat"

### Run next_pass
use next_pass to predict the overpasses of the above satellites over the selected location

In [7]:
print("*** ",sat1," ***")
result1 = next_pass.find_next_overpass(lat_S,lat_N,lon_W,lon_E,location_file_path,sat1)
# result1 is a dictionary 
s1_next_collect_info = result1.get("next_collect_info", "No collection info available")
s1_next_collect_geometry = result1.get("next_collect_geometry", None)
print(s1_next_collect_info)

***  sentinel-1  ***
+-----+--------------------------+------------------+
|   # | Collection Date & Time   |   Relative Orbit |
+=====+==========================+==================+
|   1 | 2025-03-15 01:58:22      |              137 |
+-----+--------------------------+------------------+
|   2 | 2025-03-15 13:56:57      |              144 |
+-----+--------------------------+------------------+
|   3 | 2025-03-22 01:50:13      |               64 |
+-----+--------------------------+------------------+
|   4 | 2025-03-22 13:49:51      |               71 |
+-----+--------------------------+------------------+
|   5 | 2025-03-27 01:58:22      |              137 |
+-----+--------------------------+------------------+
|   6 | 2025-03-27 13:56:57      |              144 |
+-----+--------------------------+------------------+


In [8]:
print("*** ",sat2," ***")
result2 = next_pass.find_next_overpass(lat_S,lat_N,lon_W,lon_E,location_file_path,sat2)
s2_next_collect_info = result2.get("next_collect_info", "No collection info available")
s2_next_collect_geometry = result2.get("next_collect_geometry", None)
print(s2_next_collect_info)

***  sentinel-2  ***
+-----+--------------------------+------------------+
|   # | Collection Date & Time   |   Relative Orbit |
+=====+==========================+==================+
|   1 | 2025-03-15 18:38:22      |               56 |
+-----+--------------------------+------------------+


In [9]:
print("*** ",sat3," ***")
result3 = next_pass.find_next_overpass(lat_S,lat_N,lon_W,lon_E,'',sat3)

***  landsat  ***
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Direction   |   Path |   Row | Mission   | Next Passes                                           |
+=============+========+=======+===========+=======================================================+
| Ascending   |    141 |   206 | Landsat_8 | 3/24/2025, 4/9/2025, 4/25/2025, 5/11/2025, 5/27/2025  |
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Ascending   |    141 |   206 | Landsat_9 | 3/16/2025, 4/1/2025, 4/17/2025, 5/3/2025, 5/19/2025   |
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Ascending   |    140 |   206 | Landsat_8 | 3/17/2025, 4/2/2025, 4/18/2025, 5/4/2025, 5/20/2025   |
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Ascending   |    140 |   206 | Landsat_9 | 3/25/2025, 4/10/2025, 4/26/2

### Overpasses Vizualisation  
The below vizualization tool shows the path of a selected satellite at the predicted date/time

In [11]:
# Start by choosing what satellite to visualize 
sat_to_visualize = 'Sentinel-1'; # can be Sentinel-1 or Sentinel-2

if location_file_path:
    area_polygon = next_pass.create_polygon_from_kml(location_file_path)
    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': [area_polygon]}, crs="EPSG:4326")  # WGS 84 CRS
    # Create a Folium map centered at the bounding box centroid
    m = folium.Map(location=[area_polygon.centroid.y, area_polygon.centroid.x], zoom_start=4)
    # Add the bounding box as a GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="Area of interest", style_function=style_function).add_to(m)
elif lat_S == lat_N and lon_W == lon_E:
    # Create the point
    point = Point(lon_E, lat_N)

    # Create a Folium map centered at the point location
    m = folium.Map(location=[point.y, point.x], zoom_start=4)

    # Add a cross-shaped marker to the map
    folium.Marker(
        location=[point.y, point.x],  # Latitude, Longitude
        icon=folium.Icon(icon='glyphicon-remove', icon_color='red', prefix='glyphicon')  # Cross symbol with red color
    ).add_to(m)
else:
    # Create the bounding box as a polygon
    bounding_box = box(lon_W, lat_S, lon_E, lat_N)

    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs="EPSG:4326")  # WGS 84 CRS

    # Create a Folium map centered at the bounding box centroid
    m = folium.Map(location=[bounding_box.centroid.y, bounding_box.centroid.x], zoom_start=4)
    # Add the bounding box as a GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="Bounding Box", style_function=style_function).add_to(m)

if sat_to_visualize == 'Sentinel-1':
    vi_next_collect_info = s1_next_collect_info
    vi_next_collect_geometry = s1_next_collect_geometry
elif sat_to_visualize == 'Sentinel-2':
    vi_next_collect_info = s2_next_collect_info
    vi_next_collect_geometry = s2_next_collect_geometry
else:
    vi_next_collect_info = l8_next_collect_info
    vi_next_collect_geometry = l8_next_collect_geometry
        
print('\n ** Visualizing overpasses for ',sat_to_visualize,' ** \n')
# Add each Polygon in next_collect_geometry
lines = vi_next_collect_info.split("\n")
# Clean lines by keeping only those that contain numbers (1-9)
cleaned_info = [line for line in lines if re.search(r'[1-9]', line)]  # Line must contain digits (1-9)
vi_next_collect_info_list = cleaned_info  # Now it's a list of strings (one per row in the table)
num_polygons = len(vi_next_collect_geometry)
num_info_lines = len(vi_next_collect_info_list)
#print(num_polygons)
#print(num_info_lines)

# Use the HSL distinct colors function
distinct_colors_list_1 = spread_rgb_colors(num_polygons)
distinct_colors_list_2 = hsl_distinct_colors(num_polygons)
distinct_colors_list_3 = hsl_distinct_colors_improved(num_polygons)

if vi_next_collect_geometry:
    for i, (polygon, info) in enumerate(zip(vi_next_collect_geometry, vi_next_collect_info_list), start=1):
        if isinstance(polygon, Polygon):  # Ensure it's a valid Polygon
            # Get a distinct color for each polygon
            color = distinct_colors_list_3[i - 1]

            # Print the info with corresponding color in the console
            print_colored_text(f"{info}", tuple(int(color[i:i+2], 16) for i in (1, 3, 5)))

            
            geojson_data = gpd.GeoSeries([polygon]).__geo_interface__
            folium.GeoJson(
                geojson_data, 
                name="Next Collect Area",
                style_function=lambda x, color=color: {"color": color, "weight": 2, "fillOpacity": 0.3},
                popup=folium.Popup(f"Polygon: {info}", max_width=300)  # Display corresponding info line
            ).add_to(m)

print('')
# Display the map and save to file
m.save("bounding_box_map.html")
m  # If using Jupyter Notebook


 ** Visualizing overpasses for  Sentinel-1  ** 

|   1 | 2025-03-15 01:58:22      |              137 |
|   2 | 2025-03-15 13:56:57      |              144 |
|   3 | 2025-03-22 01:50:13      |               64 |
|   4 | 2025-03-22 13:49:51      |               71 |
|   5 | 2025-03-27 01:58:22      |              137 |
|   6 | 2025-03-27 13:56:57      |              144 |

